In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np 

Current working directory is now:  C:\Users\Caroline Wang\OneDrive\Duke\Criminal Recidivism\psa-analysis


#### Read in data 



In [14]:
kentucky_data = pd.read_csv("kentucky/data/processed/kentucky_data.csv")

decoders = {"sex": {0: "male",
                    1: "female"}, 
            "race": {"White": "Caucasian",
                     "Black": "African-American",
                     "Race Unknown": "Other"} # indian or native american?
            }

In [35]:
## kentucky
kentucky_attrs = (kentucky_data[['person_id', 'screening_date', 'sex', 'race']]
                .replace(decoders))
kentucky_attrs = (pd.melt(kentucky_attrs, id_vars =['person_id', 'screening_date'], value_vars=['sex', 'race'])
                .rename({'variable': 'Attribute',
                         'value': 'Attribute Value'},
                        axis=1))

kentucky_attrs = kentucky_attrs.groupby(["Attribute", "Attribute Value"]).size().to_frame('n_inds').reset_index()

# check totals are same for race and sex
assert (kentucky_attrs[kentucky_attrs['Attribute']=='race']['n_inds'].sum()) == (kentucky_attrs[kentucky_attrs['Attribute']=='sex']['n_inds'].sum())
total_inds = kentucky_attrs[kentucky_attrs['Attribute']=='race']['n_inds'].sum()

# add percent col
kentucky_attrs["% total"] = ((kentucky_attrs['n_inds'] / total_inds) * 100).apply(lambda x: round(x, 2))
kentucky_attrs

,Attribute,Attribute Value,n_inds,% total
0,race,African-American,25367,17.37
1,race,Asian,416,0.28
2,race,Caucasian,117627,80.56
3,race,Indian,117,0.08
4,race,Other,2476,1.70
5,sex,female,101345,69.41
6,sex,male,44658,30.59


In [37]:
## write results
kentucky_attrs.to_csv("kentucky/logs/fairness_results/kentucky_racial_distr.csv")